In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
from tensorflow import keras
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


from input_generator import load_dataset, load_dataset_with_lables
from data_parser import get_tags_and_labels, get_vocab, get_tags

d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [2]:
# Get dataframe with tags and category_id columns
csvpath = 'data/CAvideos.cvs'
tab_dataframe = get_tags_and_labels(csvpath)

# Creating different coloumn based on different tags
new = tab_dataframe["tags"].str.split(".", expand = True)

max_of_tags = 15

for i in range(0, max_of_tags):
    name = "tags"+str(i)
    tab_dataframe[name] = new[i]

# Dropping old tags columns 
tab_dataframe.drop(columns =["tags"], inplace = True) 
tab_dataframe.fillna("notags", inplace = True) 


tab_dataframe.head()

,category_id,tags0,tags1,tags2,tags3,tags4,tags5,tags6,tags7,tags8,tags9,tags10,tags11,tags12,tags13,tags14
0,10,Eminem,Walk,On,Water,Aftermath/Shady/Interscope,Rap,notags,notags,notags,notags,notags,notags,notags,notags,notags
1,23,plush,bad unboxing,unboxing,fan mail,idubbbztv,idubbbztv2,things,best,packages,plushies,chontent chop,notags,notags,notags,notags
2,23,racist superman,rudy,mancuso,king,bach,racist,superman,love,rudy mancuso poo bear black white official mus...,iphone x by pineapple,lelepons,hannahstocking,rudymancuso,inanna,anwar
3,24,ryan,higa,higatv,nigahiga,i dare you,idy,rhpc,dares,no truth,comments,comedy,funny,stupid,fail,notags
4,10,edsheeran,ed sheeran,acoustic,live,cover,official,remix,official video,lyrics,session,notags,notags,notags,notags,notags


In [3]:
# Getting leaning vocab
vocab = get_vocab(get_tags(csvpath))
voc_di = {i:vo for i,vo in enumerate(vocab)}
voc_di[0] = 'notags'
voc_di = {vo:i for i, vo in voc_di.items()}

In [4]:
# Mapping tags to indexes in vocab
for index in range(0, max_of_tags):
    tab_dataframe['tags{}'.format(index)] = tab_dataframe['tags{}'.format(index)].map(voc_di)
    
tab_dataframe.head()

,category_id,tags0,tags1,tags2,tags3,tags4,tags5,tags6,tags7,tags8,tags9,tags10,tags11,tags12,tags13,tags14
0,10,94955.0,121932.0,138828.0,83019.0,57001.0,146235.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,23,137849.0,95336.0,122616.0,82492.0,19550.0,143208.0,104727.0,9050.0,128149.0,61709.0,104415.0,0.0,0.0,0.0,0.0
2,23,142083.0,10650.0,94348.0,132625.0,76677.0,125083.0,149199.0,142855.0,21400.0,114184.0,50974.0,30466.0,59089.0,126398.0,43670.0
3,24,96987.0,149430.0,137613.0,82936.0,78080.0,21542.0,148662.0,70282.0,150319.0,115526.0,105695.0,12884.0,138969.0,54052.0,0.0
4,10,92124.0,27964.0,140773.0,23383.0,150739.0,31028.0,89541.0,102497.0,93981.0,15352.0,0.0,0.0,0.0,0.0,0.0


In [5]:
for column in tab_dataframe.columns:
    if column == 'category_id':
        pass
    else:
        if tab_dataframe[column].max() == 0:
            tab_dataframe[column] = tab_dataframe[column]

        else:
            tab_dataframe[column] = tab_dataframe[column] / tab_dataframe[column].max()

tab_dataframe.head()

,category_id,tags0,tags1,tags2,tags3,tags4,tags5,tags6,tags7,tags8,tags9,tags10,tags11,tags12,tags13,tags14
0,10,0.626741,0.804864,0.916411,0.547987,0.376254,0.965197,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,23,0.909858,0.629305,0.809395,0.544509,0.129046,0.945217,0.691263,0.059733,0.845823,0.407301,0.689153,0.000000,0.000000,0.000000,0.000000
2,23,0.937804,0.070300,0.622796,0.875424,0.506132,0.825587,0.984805,0.942888,0.141247,0.753655,0.336435,0.201105,0.390026,0.834316,0.288247
3,24,0.640153,0.986376,0.908391,0.547440,0.515393,0.142184,0.981261,0.463883,0.992152,0.762513,0.697602,0.085047,0.917287,0.356781,0.000000
4,10,0.608055,0.184588,0.929250,0.154345,0.995003,0.204794,0.591026,0.676512,0.620304,0.101329,0.000000,0.000000,0.000000,0.000000,0.000000


In [6]:
# Create train,validation and test examples from dataframe
# train_ds, val_ds, test_ds = load_dataset(tab_dataframe)
train_ds, test_ds, train_lb, test_lb = load_dataset_with_lables(tab_dataframe, 'category_id')


In [7]:
train_ds.values

array([[0.09310522, 0.71843769, 0.99899004, ..., 0.0850429 , 0.21745358,
        0.0898074 ],
       [0.5165538 , 0.83565686, 0.61161389, ..., 0.        , 0.        ,
        0.        ],
       [0.8311816 , 0.85092479, 0.32739899, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.30107058, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.67918762, 0.25383183, 0.18411655, ..., 0.94829703, 0.03330715,
        0.18563451],
       [0.30070096, 0.3954216 , 0.1618578 , ..., 0.94040264, 0.57702691,
        0.90122903]])

In [13]:
model = tf.keras.Sequential([
    layers.Flatten(),
    layers.Dense(64, kernel_regularizer=keras.regularizers.l2(0.01), input_shape = (15,)),
    layers.Dropout(0.5),
    layers.Dense(128, kernel_regularizer=keras.regularizers.l2(0.01)),
    layers.Dense(64, activation='sigmoid'),
    layers.Dense(44, activation='softmax', kernel_regularizer=keras.regularizers.l2(0.01))
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', 'binary_crossentropy'],
              )

history = model.fit(train_ds.values, train_lb.values,
          batch_size=500,
          epochs=5)


Train on 32704 samples
Epoch 1/5
32704/32704 [==============================] - 1s 19us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 2/5
32704/32704 [==============================] - 0s 10us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 3/5
32704/32704 [==============================] - 0s 9us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 4/5
32704/32704 [==============================] - 0s 10us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
Epoch 5/5
32704/32704 [==============================] - 0s 10us/sample - loss: nan - accuracy: 0.0000e+00 - binary_crossentropy: nan
